In [1]:
import os
from chainbase_sdk.sql_alpha import ChainbaseSQLAlpha

CHAINBASE_API_KEY=os.environ['CHAINBASE_API_KEY']

L2_CHAIN = 'base'
L1_PORTAL = '0x49048044d57e1c92a77f79988d21fa8faf74e97e'

In [2]:
client = ChainbaseSQLAlpha(api_key=CHAINBASE_API_KEY)

In [3]:
def withdrawal_basic_query(client, chain, contract_address):
    query = f"""
WITH initiated_withdrawals AS (
    SELECT 
        block_number, 
        block_timestamp, 
        transaction_hash,
        CAST(CAST(decoded AS JSON) -> '$[1]' AS VARCHAR) AS sender,
        CAST(CAST(decoded AS JSON) -> '$[2]' AS VARCHAR) AS target,
        CAST(CAST(CAST(decoded AS JSON) -> '$[3]' AS VARCHAR) AS BIGINT) AS value,
        CAST(CAST(decoded AS JSON) -> '$[6]' AS VARCHAR) AS withdrawal_hash
    FROM 
        {chain}.transaction_logs_decoded
    WHERE 
        contract_address = '0x4200000000000000000000000000000000000016'
        AND CAST(CAST(CAST(decoded AS JSON) -> '$[3]' AS VARCHAR) AS BIGINT) <> 0
),
proven_withdrawals AS (
    SELECT 
        block_number, 
        block_timestamp, 
        transaction_hash, 
        CAST(CAST(decoded AS JSON) -> '$[0]' AS VARCHAR) AS withdrawal_hash 
    FROM 
        ethereum.transaction_logs_decoded 
    WHERE 
        contract_address = '{contract_address}' 
        AND method_id = '0x67a6208cfcc0801d50f6cbe764733f4fddf66ac0b04442061a8a8c0cb6b63f62' -- WithdrawalProven
),
finalized_withdrawals AS (
    SELECT 
        block_number, 
        block_timestamp, 
        transaction_hash, 
        CAST(CAST(decoded AS JSON) -> '$[0]' AS VARCHAR) AS withdrawal_hash 
    FROM 
        ethereum.transaction_logs_decoded 
    WHERE 
        contract_address = '{contract_address}' 
        AND method_id = '0xdb5c7652857aa163daadd670e116628fb42e869d8ac4251ef8971d9e5727df1b' -- WithdrawalFinalized
),
aggregate_data AS (
    SELECT 
        iw.block_number AS initiated_block_number,
        iw.block_timestamp AS initiated_block_timestamp,
        iw.transaction_hash AS initiated_transaction_hash,
        iw.sender,
        iw.target,
        iw.value,
        iw.withdrawal_hash AS initiated_withdrawal_hash,
        pw.block_number AS proven_block_number,
        pw.block_timestamp AS proven_block_timestamp,
        pw.transaction_hash AS proven_transaction_hash,
        pw.withdrawal_hash AS proven_withdrawal_hash,
        fw.block_number AS finalized_block_number,
        fw.block_timestamp AS finalized_block_timestamp,
        fw.transaction_hash AS finalized_transaction_hash,
        fw.withdrawal_hash AS finalized_withdrawal_hash
    FROM 
        initiated_withdrawals iw
    LEFT JOIN 
        proven_withdrawals pw ON iw.withdrawal_hash = pw.withdrawal_hash
    LEFT JOIN 
        finalized_withdrawals fw ON iw.withdrawal_hash = fw.withdrawal_hash
)
SELECT * FROM aggregate_data
    """
    # missing and success = true

    df_l1_finalized = client.query_pandas(query)

    return df_l1_finalized

In [4]:
def deposit_basic_query(client, contract_address):
    query = f"""
    SELECT 
        block_number, 
        block_timestamp, 
        transaction_hash,
        CAST(CAST(decoded AS JSON) -> '$[0]' AS VARCHAR) AS sender,
        CAST(CAST(decoded AS JSON) -> '$[1]' AS VARCHAR) AS target,
        CAST(CONV(SUBSTR(CAST(decoded AS JSON) -> '$[3]', 3, 64), 16, 10) AS BIGINT) AS value
    FROM 
        ethereum.transaction_logs_decoded 
    WHERE 
        contract_address = '{contract_address}' 
        AND method_id = '0xb3813568d9991fc951961fcb4c784893574240a28925604d09fc577c55bb7c32' -- TransactionDeposited
        AND CONV(SUBSTR(CAST(decoded AS JSON) -> '$[3]', 3, 64), 16, 10) <> 0 -- exclude zero value messages
    """
    df_deposit = client.query_pandas(query)

    return df_deposit

In [5]:
base_withdrawals = withdrawal_basic_query(client=client, chain=L2_CHAIN,contract_address=L1_PORTAL)

len(base_withdrawals)

10000

In [6]:
base_withdrawals.head()

,initiated_block_number,initiated_block_timestamp,initiated_transaction_hash,sender,target,value,initiated_withdrawal_hash,proven_block_number,proven_block_timestamp,proven_transaction_hash,proven_withdrawal_hash,finalized_block_number,finalized_block_timestamp,finalized_transaction_hash,finalized_withdrawal_hash
0,11183443,2024-02-28 21:37:13,0x532bc896197eafa7ff2d1fde2ff92f82e4afa1a26847...,0x6ff755b484133ecfab8b973b2d6fbdd5ee881176,0x6ff755b484133ecfab8b973b2d6fbdd5ee881176,50000000000000000,0x5ad2fcf8b279ef88df4193952710c4b0032b610aac6f...,19329885,2024-02-29 01:24:35,0xb73a9215fd1af942d6cf78a2ef37f0d4e1ed3ff0db13...,0x5ad2fcf8b279ef88df4193952710c4b0032b610aac6f...,19381381,2024-03-07 06:03:59,0x0e94c1f74524a458141e01a76729e0cdb0efe0027c1a...,0x5ad2fcf8b279ef88df4193952710c4b0032b610aac6f...
1,9908528,2024-01-30 09:20:03,0xbff14e32363a82cf5cec8e960a772f6e921da922d936...,0xfdfbb58507d5eda67ab9d5f3000781ee335d9f15,0xfdfbb58507d5eda67ab9d5f3000781ee335d9f15,244871000000000000,0x173e6b15b0217d556fb121cfafda6bce5ca4f59d9935...,19323554,2024-02-28 04:08:23,0xac68287e9e5719f8ceda18806f7b102599441f5beccf...,0x173e6b15b0217d556fb121cfafda6bce5ca4f59d9935...,19382576,2024-03-07 10:03:59,0xb7f12ca8a02f0a2d6c534c88a125429c74e930d8a8b0...,0x173e6b15b0217d556fb121cfafda6bce5ca4f59d9935...
2,11216185,2024-02-29 15:48:37,0x6121faf2b9d0127f53a00beba8a9f772c18659e9472f...,0xa99d7a5693318a5b6a732871075df0826629b119,0xa99d7a5693318a5b6a732871075df0826629b119,1880000000000000000,0xe3fdd1a6a0e767b651b5f77208ef938b6b95742a428e...,19334520,2024-02-29 17:00:47,0x58137b8c067cd343f964af2d0d5e6d0cca9492b515db...,0xe3fdd1a6a0e767b651b5f77208ef938b6b95742a428e...,19384679,2024-03-07 17:06:59,0xd3375d4d7632fbf6006bb89eead90ef40355c16a18b2...,0xe3fdd1a6a0e767b651b5f77208ef938b6b95742a428e...
3,11162126,2024-02-28 09:46:39,0xffdd82a3c223698055a73c2a7d52e002498ccdff4850...,0x8217233c9eb365667817255cb84999761981e95c,0x8217233c9eb365667817255cb84999761981e95c,776035000000000000,0x552754daed4f418b04077a5eb52d85c69e62793cf123...,19338372,2024-03-01 05:55:47,0xc0ea92adec519ed51fc9caa8319890995962042615e0...,0x552754daed4f418b04077a5eb52d85c69e62793cf123...,19388502,2024-03-08 05:56:35,0x9abf56a573fae5ec7ffc1b1bee3398b13ea8f6cf013e...,0x552754daed4f418b04077a5eb52d85c69e62793cf123...
4,11473152,2024-03-06 14:34:11,0x5af77bf9608fab4542dbcffa672380a64c453a26767e...,0x695d71b71102c7097e77a7e4eaa158b3735f382c,0x695d71b71102c7097e77a7e4eaa158b3735f382c,2292894000000000000,0xa4bc037df2d670007c4f685677a907db925d88efaaf0...,19377762,2024-03-06 17:56:23,0x6abe111bb72045f7a360645aecfd1f53538fbbeffa2b...,0xa4bc037df2d670007c4f685677a907db925d88efaaf0...,19427797,2024-03-13 18:02:47,0x7723218b917e5d7ad13fd8ecdef60236bb04e007a288...,0xa4bc037df2d670007c4f685677a907db925d88efaaf0...


In [7]:
base_deposits = deposit_basic_query(client=client, contract_address=L1_PORTAL)

len(base_deposits)

10000

In [8]:
base_deposits.head()

,block_number,block_timestamp,transaction_hash,sender,target,value
0,18545672,2023-11-11 02:19:23,0x42f118963d310ff710c0223c16eaae5885b9876ea1e3...,0x977f82a600a1414e583f7f13623f1ac5d58b1c0b,0x4200000000000000000000000000000000000007,162000000000000000
1,18545706,2023-11-11 02:26:11,0xb7001b79a247b546a1139ada80234197eb1dff3580d7...,0x977f82a600a1414e583f7f13623f1ac5d58b1c0b,0x4200000000000000000000000000000000000007,75000000000000000
2,18545718,2023-11-11 02:28:35,0x22c2b4998dce74582f01ae2178cde2432afb0adfa629...,0x2b9db3fb3ae561b8dab07dd57d8aaa7d3ed5a3f3,0x2b9db3fb3ae561b8dab07dd57d8aaa7d3ed5a3f3,8704780000000000
3,18545721,2023-11-11 02:29:11,0xab599d86eb18a0e7f13d31b27632fe722f31171d0d9b...,0xf1ba5efe2d0e9b8e59f80956caef65db9d0349bd,0xf1ba5efe2d0e9b8e59f80956caef65db9d0349bd,6806000000000000000
4,18545759,2023-11-11 02:36:47,0x006ee0aa5c54aa9da5275ff0abdd9c4d9b6c0bd0a181...,0xb40bb7bca9f5d31529839379908a4ae40bb27832,0xb40bb7bca9f5d31529839379908a4ae40bb27832,163000000000000000


In [9]:
# save to csv

base_withdrawals.to_csv(f'../../data/{L2_CHAIN}/basic_withdrawals.csv', index=False)
base_deposits.to_csv(f'../../data/{L2_CHAIN}/basic_deposits.csv', index=False)

## tables for the frontend
as the api limits to 10_000 rows, it is necessary for me to manipulate the data directly within sql. hence i have made three queries to extract all views i am interested in for each chain.

In [10]:
def counters(chain, l1_portal_address, client = client):
    query = f"""
WITH initiated_withdrawals AS (
    SELECT 
        block_number, 
        block_timestamp, 
        transaction_hash,
        CAST(CAST(decoded AS JSON) -> '$[1]' AS VARCHAR) AS sender,
        CAST(CAST(decoded AS JSON) -> '$[2]' AS VARCHAR) AS target,
        CAST(CAST(CAST(decoded AS JSON) -> '$[3]' AS VARCHAR) AS DOUBLE)/1e18 AS value,
        CAST(CAST(decoded AS JSON) -> '$[6]' AS VARCHAR) AS withdrawal_hash
    FROM 
        {chain}.transaction_logs_decoded
    WHERE 
        contract_address = '0x4200000000000000000000000000000000000016'
        AND CAST(CAST(CAST(decoded AS JSON) -> '$[3]' AS VARCHAR) AS DOUBLE) <> 0
),
proven_withdrawals AS (
    SELECT 
        block_number, 
        block_timestamp, 
        transaction_hash, 
        CAST(CAST(decoded AS JSON) -> '$[0]' AS VARCHAR) AS withdrawal_hash 
    FROM 
        ethereum.transaction_logs_decoded 
    WHERE 
        contract_address = '{l1_portal_address}' 
        AND method_id = '0x67a6208cfcc0801d50f6cbe764733f4fddf66ac0b04442061a8a8c0cb6b63f62' -- WithdrawalProven
),
finalized_withdrawals AS (
    SELECT 
        block_number, 
        block_timestamp, 
        transaction_hash, 
        CAST(CAST(decoded AS JSON) -> '$[0]' AS VARCHAR) AS withdrawal_hash 
    FROM 
        ethereum.transaction_logs_decoded 
    WHERE 
        contract_address = '{l1_portal_address}' 
        AND method_id = '0xdb5c7652857aa163daadd670e116628fb42e869d8ac4251ef8971d9e5727df1b' -- WithdrawalFinalized
),
aggregate_data AS (
    SELECT 
        iw.block_number AS initiated_block_number,
        iw.block_timestamp AS initiated_block_timestamp,
        iw.transaction_hash AS initiated_transaction_hash,
        iw.sender,
        iw.target,
        iw.value,
        iw.withdrawal_hash AS initiated_withdrawal_hash,
        pw.block_number AS proven_block_number,
        pw.block_timestamp AS proven_block_timestamp,
        pw.transaction_hash AS proven_transaction_hash,
        pw.withdrawal_hash AS proven_withdrawal_hash,
        fw.block_number AS finalized_block_number,
        fw.block_timestamp AS finalized_block_timestamp,
        fw.transaction_hash AS finalized_transaction_hash,
        fw.withdrawal_hash AS finalized_withdrawal_hash
    FROM 
        initiated_withdrawals iw
    LEFT JOIN 
        proven_withdrawals pw ON iw.withdrawal_hash = pw.withdrawal_hash
    LEFT JOIN 
        finalized_withdrawals fw ON iw.withdrawal_hash = fw.withdrawal_hash
)
SELECT
    SUM(value) AS total_value,
    SUM(IF(finalized_transaction_hash IS NULL, value, 0)) AS sum_if_finalized_null,
    SUM(IF(proven_withdrawal_hash IS NULL, value, 0)) AS sum_if_proven_null,
    SUM(IF(DATEDIFF(NOW(), initiated_block_timestamp) > 14 and proven_withdrawal_hash is null, value, 0)) AS not_proven_2w,
    SUM(IF(DATEDIFF(NOW(), initiated_block_timestamp) > 28 and proven_withdrawal_hash is null, value, 0)) AS not_proven_4w,
    SUM(IF(DATEDIFF(NOW(), initiated_block_timestamp) > 14 AND finalized_transaction_hash IS NULL, value, 0)) AS not_finalized_2w,
    SUM(IF(DATEDIFF(NOW(), initiated_block_timestamp) > 28 AND finalized_transaction_hash IS NULL, value, 0)) AS not_finalized_4w    
FROM 
    aggregate_data    
"""
    df = client.query_pandas(query)
    return df

In [11]:
df_counters = counters(chain=L2_CHAIN, l1_portal_address=L1_PORTAL)

NOT EXISTS:  DOUBLE
NOT EXISTS:  DOUBLE
NOT EXISTS:  DOUBLE
NOT EXISTS:  DOUBLE
NOT EXISTS:  DOUBLE
NOT EXISTS:  DOUBLE
NOT EXISTS:  DOUBLE


In [12]:
df_counters

,total_value,sum_if_finalized_null,sum_if_proven_null,not_proven_2w,not_proven_4w,not_finalized_2w,not_finalized_4w
0,674332.805346,38046.695982,5312.300236,4462.931718,4454.099166,19991.533587,19971.289809


In [13]:
df_counters.to_csv(f'../../data/{L2_CHAIN}/counters.csv', index=False)

In [14]:
def daily_volumes(chain, l1_portal_address, client = client):
    query = f"""
WITH initiated_withdrawals AS (
    SELECT 
        block_number, 
        block_timestamp, 
        transaction_hash,
        CAST(CAST(decoded AS JSON) -> '$[1]' AS VARCHAR) AS sender,
        CAST(CAST(decoded AS JSON) -> '$[2]' AS VARCHAR) AS target,
        CAST(CAST(CAST(decoded AS JSON) -> '$[3]' AS VARCHAR) AS DOUBLE) / 1e18 AS value,
        CAST(CAST(decoded AS JSON) -> '$[6]' AS VARCHAR) AS withdrawal_hash
    FROM 
        {chain}.transaction_logs_decoded
    WHERE 
        contract_address = '0x4200000000000000000000000000000000000016'
        AND CAST(CAST(CAST(decoded AS JSON) -> '$[3]' AS VARCHAR) AS DOUBLE) <> 0
),
proven_withdrawals AS (
    SELECT 
        iw.block_timestamp, 
        iw.withdrawal_hash,
        iw.value AS initiated_value
    FROM 
        initiated_withdrawals iw
    JOIN 
        ethereum.transaction_logs_decoded pw ON iw.withdrawal_hash = CAST(CAST(pw.decoded AS JSON) -> '$[0]' AS VARCHAR)
    WHERE 
        pw.contract_address = '{l1_portal_address}' 
        AND pw.method_id = '0x67a6208cfcc0801d50f6cbe764733f4fddf66ac0b04442061a8a8c0cb6b63f62' -- WithdrawalProven
),
finalized_withdrawals AS (
    SELECT 
        iw.block_timestamp, 
        iw.withdrawal_hash,
        iw.value AS initiated_value
    FROM 
        initiated_withdrawals iw
    JOIN 
        ethereum.transaction_logs_decoded fw ON iw.withdrawal_hash = CAST(CAST(fw.decoded AS JSON) -> '$[0]' AS VARCHAR)
    WHERE 
        fw.contract_address = '{l1_portal_address}' 
        AND fw.method_id = '0xdb5c7652857aa163daadd670e116628fb42e869d8ac4251ef8971d9e5727df1b' -- WithdrawalFinalized
),
deposits_basic AS (
    SELECT 
        block_number, 
        block_timestamp, 
        transaction_hash,
        CAST(CAST(decoded AS JSON) -> '$[0]' AS VARCHAR) AS sender,
        CAST(CAST(decoded AS JSON) -> '$[1]' AS VARCHAR) AS target,
        CAST(CONV(SUBSTR(CAST(decoded AS JSON) -> '$[3]', 3, 64), 16, 10) AS DOUBLE) / 1e18 AS value
    FROM 
        ethereum.transaction_logs_decoded 
    WHERE 
        contract_address = '{l1_portal_address}' 
        AND method_id = '0xb3813568d9991fc951961fcb4c784893574240a28925604d09fc577c55bb7c32' -- TransactionDeposited
        AND CONV(SUBSTR(CAST(decoded AS JSON) -> '$[3]', 3, 64), 16, 10) <> 0 -- exclude zero value messages
),
aggregated_data AS (
    SELECT 
        DATE(iw.block_timestamp) AS date,
        SUM(iw.value) AS initiated_volume,
        COALESCE(SUM(pw.initiated_value), 0) AS proved_volume,
        COALESCE(SUM(fw.initiated_value), 0) AS finalized_volume,
        COALESCE(SUM(dep.value), 0) AS deposited_volume
    FROM 
        initiated_withdrawals iw
    LEFT JOIN 
        proven_withdrawals pw ON iw.withdrawal_hash = pw.withdrawal_hash
    LEFT JOIN 
        finalized_withdrawals fw ON iw.withdrawal_hash = fw.withdrawal_hash
    LEFT JOIN 
        deposits_basic dep ON DATE(iw.block_timestamp) = DATE(dep.block_timestamp)
    GROUP BY 
        DATE(iw.block_timestamp)
),
cumulative_data AS (
    SELECT 
        date,
        initiated_volume,
        proved_volume,
        finalized_volume,
        deposited_volume,
        (COALESCE(deposited_volume, 0) - COALESCE(initiated_volume, 0)) AS daily_difference,
        SUM(initiated_volume) OVER (ORDER BY date) AS cumulative_initiated,
        SUM(proved_volume) OVER (ORDER BY date) AS cumulative_proved,
        SUM(finalized_volume) OVER (ORDER BY date) AS cumulative_finalized,
        SUM(deposited_volume) OVER (ORDER BY date) AS cumulative_deposited
    FROM 
        aggregated_data
)
SELECT 
    date,
    initiated_volume,
    proved_volume,
    finalized_volume,
    deposited_volume,
    daily_difference,
    cumulative_initiated,
    cumulative_proved,
    cumulative_finalized,
    cumulative_deposited
FROM 
    cumulative_data
ORDER BY 
    date
 
"""
    df = client.query_pandas(query)
    return df

In [15]:
df_daily = daily_volumes(chain=L2_CHAIN, l1_portal_address='0x49048044d57e1c92a77f79988d21fa8faf74e97e')

NOT EXISTS:  DATE
NOT EXISTS:  DOUBLE
NOT EXISTS:  DOUBLE
NOT EXISTS:  DOUBLE
NOT EXISTS:  DOUBLE
NOT EXISTS:  DOUBLE
NOT EXISTS:  DOUBLE
NOT EXISTS:  DOUBLE
NOT EXISTS:  DOUBLE
NOT EXISTS:  DOUBLE


In [16]:
print(df_daily)
df_daily.to_csv(f'../../data/{L2_CHAIN}/daily_volumes.csv', index=False)

           date  initiated_volume  proved_volume  finalized_volume  \
0    2023-07-11      3.703704e-01   0.000000e+00          0.000000   
1    2023-07-12      2.345679e-02   2.345679e-02          0.023457   
2    2023-07-27      2.272500e+00   0.000000e+00          0.000000   
3    2023-07-28      1.561460e+01   0.000000e+00          0.000000   
4    2023-07-30      2.451511e+03   9.477560e+02        462.320000   
..          ...               ...            ...               ...   
388  2024-10-14      1.440440e+06   1.439431e+06          0.000000   
389  2024-10-15      1.885532e+06   1.885171e+06          0.000000   
390  2024-10-16      2.243050e+05   2.240612e+05          0.000000   
391  2024-10-17      1.920407e+05   1.920083e+05          0.000000   
392  2024-10-18      1.519224e+06   1.253740e+06          0.000000   

     deposited_volume  daily_difference  cumulative_initiated  \
0            0.750000      3.796296e-01          3.703704e-01   
1            0.000000     -2.

In [17]:
def stuck_in_contract(chain, l1_portal_address, client = client):
    query = f"""
WITH initiated_withdrawals AS (
    SELECT 
        block_number, 
        block_timestamp, 
        transaction_hash,
        CAST(CAST(decoded AS JSON) -> '$[1]' AS VARCHAR) AS sender,
        CAST(CAST(decoded AS JSON) -> '$[2]' AS VARCHAR) AS target,
        CAST(CAST(CAST(decoded AS JSON) -> '$[3]' AS VARCHAR) AS DOUBLE) / 1e18 AS value,
        CAST(CAST(decoded AS JSON) -> '$[6]' AS VARCHAR) AS withdrawal_hash
    FROM 
        {chain}.transaction_logs_decoded
    WHERE
        contract_address = '0x4200000000000000000000000000000000000016'
        AND CAST(CAST(CAST(decoded AS JSON) -> '$[3]' AS VARCHAR) AS DOUBLE) <> 0
),
proven_withdrawals AS (
    SELECT 
        iw.block_timestamp, 
        iw.withdrawal_hash,
        iw.value AS initiated_value
    FROM 
        initiated_withdrawals iw
    JOIN 
        ethereum.transaction_logs_decoded pw ON iw.withdrawal_hash = CAST(CAST(pw.decoded AS JSON) -> '$[0]' AS VARCHAR)
    WHERE 
        pw.contract_address = '{l1_portal_address}' 
        AND pw.method_id = '0x67a6208cfcc0801d50f6cbe764733f4fddf66ac0b04442061a8a8c0cb6b63f62' -- WithdrawalProven
),
finalized_withdrawals AS (
    SELECT 
        iw.block_timestamp, 
        iw.withdrawal_hash,
        iw.value AS initiated_value
    FROM 
        initiated_withdrawals iw
    JOIN 
        ethereum.transaction_logs_decoded fw ON iw.withdrawal_hash = CAST(CAST(fw.decoded AS JSON) -> '$[0]' AS VARCHAR)
    WHERE 
        fw.contract_address = '{l1_portal_address}' 
        AND fw.method_id = '0xdb5c7652857aa163daadd670e116628fb42e869d8ac4251ef8971d9e5727df1b' -- WithdrawalFinalized
),
withdrawals_not_withdrawn AS (
    SELECT 
        iw.withdrawal_hash,
        iw.value AS initiated_value,
        iw.block_timestamp
    FROM 
        initiated_withdrawals iw
    LEFT JOIN 
        proven_withdrawals pw ON iw.withdrawal_hash = pw.withdrawal_hash
    LEFT JOIN 
        finalized_withdrawals fw ON iw.withdrawal_hash = fw.withdrawal_hash
    WHERE 
        pw.withdrawal_hash IS NULL 
        AND fw.withdrawal_hash IS NULL
        AND iw.block_timestamp < DATE_SUB(CURRENT_DATE, INTERVAL 14 DAY)
),
daily_not_withdrawn AS (
    SELECT 
        DATE(block_timestamp) AS date,
        SUM(initiated_value) AS total_not_withdrawn
    FROM 
        withdrawals_not_withdrawn
    GROUP BY 
        DATE(block_timestamp)
)
SELECT 
    date,
    COALESCE(total_not_withdrawn, 0) AS total_not_withdrawn
FROM 
    daily_not_withdrawn
ORDER BY 
    date
"""

    df = client.query_pandas(query)
    return df

In [18]:
df_stuck = stuck_in_contract(chain=L2_CHAIN, l1_portal_address=L1_PORTAL)

NOT EXISTS:  DATE
NOT EXISTS:  DOUBLE


In [19]:
df_stuck.to_csv(f'../../data/{L2_CHAIN}/forgotten_funds.csv', index=False)